# Disaster classification

## 1. Importing the library

In [1]:
import numpy as np
import pandas as pd
import re
import emoji

## 2. Importing the dataset

In [2]:
data = pd.read_csv('/Users/savinaysingh/Downloads/tweets 2.csv')
data.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [3]:
# Loading required features
data = data[['text','target']]
data.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


In [4]:
data = data.dropna()
data.isna().sum()

text      0
target    0
dtype: int64

In [5]:
data['text'] = [entry.lower() for entry in data['text']]
data['text'].head()

0    communal violence in bhainsa, telangana. "ston...
1    telangana: section 144 has been imposed in bha...
2    arsonist sets cars ablaze at dealership https:...
3    arsonist sets cars ablaze at dealership https:...
4    "lord jesus, your love brings freedom and pard...
Name: text, dtype: object

In [6]:
# url not req while finding if text is disaster or not
def remove_urls(text):
    pattern = re.compile(r'http\S+|www\S+')
    return pattern.sub('', text)

# spl chars not req while finding if text is disaster or not
def remove_special_characters(text):
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    return pattern.sub('', text)

def remove_hashtags(text):
    pattern = re.compile(r'#\w+')
    return pattern.sub('', text)

def remove_emojis(text):
    return emoji.demojize(text)

In [7]:
def preprocess_text(text):
    text = remove_urls(text)
    text = remove_special_characters(text)
    text = remove_hashtags(text)
    text = remove_emojis(text)
    return text

In [8]:
data['text'] = data['text'].apply(preprocess_text)


In [9]:
data = data[data['text'].map(len) > 0]
class_counts = data['target'].value_counts()
print(class_counts)

target
0    9254
1    2114
Name: count, dtype: int64


In [10]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, RobertaConfig

/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/iLab-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    data['text'],
    data['target'],
    test_size=0.2,
    random_state=42
)

In [13]:
# Load pre-trained RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the input texts
train_encodings = tokenizer(train_data.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_data.tolist(), truncation=True, padding=True)

In [14]:
# Create custom dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[index])
        return item

In [15]:
# Create training and validation datasets
train_dataset = TextDataset(train_encodings, train_labels.tolist())
val_dataset = TextDataset(val_encodings, val_labels.tolist())

In [16]:
# Define the model configuration
model_config = RobertaConfig.from_pretrained('roberta-base', num_labels=2)
model_config.hidden_dropout_prob = 0.2  # Dropout probability'

# Create a data loader for training and validation
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', config=model_config)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/iLab-env/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

num_epochs = 20  # Define the number of epochs

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    predictions = []
    true_labels = []

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        _, predicted_labels = torch.max(outputs.logits, 1)
        train_total += labels.size(0)
        train_correct += (predicted_labels == labels).sum().item()

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

    train_loss = train_loss / len(train_loader)
    train_accuracy = 100.0 * train_correct / train_total
    train_f1 = f1_score(true_labels, predictions)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            _, predicted_labels = torch.max(outputs.logits, 1)
            val_total += labels.size(0)
            val_correct += (predicted_labels == labels).sum().item()

            val_predictions.extend(predicted_labels.tolist())
            val_true_labels.extend(labels.tolist())

    val_loss = val_loss / len(val_loader)
    val_accuracy = 100.0 * val_correct / val_total
    val_f1 = f1_score(val_true_labels, val_predictions)

    print(f"Epoch: {epoch + 1}")
    print(f"Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}")
    print(f"Train Accuracy: {train_accuracy:.2f}% | Validation Accuracy: {val_accuracy:.2f}%")
    print(f"Train F1-score: {train_f1:.4f} | Validation F1-score: {val_f1:.4f}")

Epoch: 1
Train Loss: 0.3088 | Validation Loss: 0.2704
Train Accuracy: 86.89% | Validation Accuracy: 88.87%
Train F1-score: 0.6019 | Validation F1-score: 0.7323
Epoch: 2
Train Loss: 0.2242 | Validation Loss: 0.2128
Train Accuracy: 90.84% | Validation Accuracy: 91.16%
Train F1-score: 0.7422 | Validation F1-score: 0.7426
Epoch: 3
Train Loss: 0.1844 | Validation Loss: 0.3383
Train Accuracy: 93.07% | Validation Accuracy: 87.95%
Train F1-score: 0.8076 | Validation F1-score: 0.7227
Epoch: 4
Train Loss: 0.1529 | Validation Loss: 0.2493
Train Accuracy: 93.93% | Validation Accuracy: 89.89%
Train F1-score: 0.8345 | Validation F1-score: 0.7450


In [ ]:
# Evaluate on the test set
test_data = pd.read_csv('/kaggle/input/disaster-tweets/tweets.csv')
test_data['text'] = test_data['text'].apply(preprocess_text)
test_encodings = tokenizer(test_data['text'].tolist(), truncation=True, padding=True)
test_dataset = TextDataset(test_encodings, test_data['target'].tolist())
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
test_predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).tolist()
        test_predictions.extend(predictions)

test_data['predicted_target'] = test_predictions
test_data[['text', 'predicted_target']].to_csv('test_predictions.csv', index=False)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

test_true_labels = test_data['target'].tolist()
test_predicted_labels = test_data['predicted_target'].tolist()

# Calculate F1-score and accuracy
test_f1 = f1_score(test_true_labels, test_predicted_labels)
test_accuracy = accuracy_score(test_true_labels, test_predicted_labels) * 100

print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Test F1-score: {test_f1:.4f}")

# Plot confusion matrix
cm = confusion_matrix(test_true_labels, test_predicted_labels)
classes = ['Non-Disaster', 'Disaster']

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', cbar=False,
            xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
# Define a list of input texts
input_texts = [
    "Communal violence in Bhainsa, Telangana. Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",
    "Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznnPNi8",
    "Juwan Johnson/Oregon is one big dude. Looks like a tight end stuck in the receiver group by accident.",
    "Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. ",
    "Kumera yellow Aftershock #PH19FOSSIL featured in bankbtn Good Living magazine Photography Stylist shino… https://t.co/V9LJ65ttzR"
]

expected = [
    "Disaster", "Not Disaster", "Not Disaster", "Disaster", "Not Disaster"
]

# Preprocess the input texts
preprocessed_texts = [preprocess_text(text) for text in input_texts]
input_encodings = tokenizer(preprocessed_texts, truncation=True, padding=True)
input_dataset = TextDataset(input_encodings, None)
input_loader = DataLoader(input_dataset, batch_size=1, shuffle=False)

model.eval()

with torch.no_grad():
    for i, batch in enumerate(input_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1).tolist()
        prediction = 'Disaster' if predictions[0] == 1 else 'Not Disaster'
        print(f"Input: {input_texts[i]}")
        print(f"Target: {expected[i]}")
        print(f"Prediction: {prediction}")
        print()